# This script try to test and rerun the Leash Tutorial
Including the the use of duckdb, ECFPs (fingerprint), and Random Forest.

In [4]:
import duckdb
import pandas as pd

In [5]:
train_path = '/mnt/isilon/wang_lab/pengwang/projects/Belka/train.parquet'
test_path = '/mnt/isilon/wang_lab/pengwang/projects/Belka/test.parquet'

con = duckdb.connect()

df = con.query(f"""(SELECT *
                        FROM parquet_scan('{train_path}')
                        WHERE binds = 0
                        ORDER BY random()
                        LIMIT 30000)
                        UNION ALL
                        (SELECT *
                        FROM parquet_scan('{train_path}')
                        WHERE binds = 1
                        ORDER BY random()
                        LIMIT 30000)""").df()

con.close()

In [9]:
df.head()

,id,buildingblock1_smiles,buildingblock2_smiles,buildingblock3_smiles,molecule_smiles,protein_name,binds
0,96833364,O=C(N[C@@H](Cc1ccc(Cl)cc1Cl)C(=O)O)OCC1c2ccccc...,Nc1ccc2c(c1)OCO2,NCC1CCCn2ccnc21,O=C(N[Dy])[C@H](Cc1ccc(Cl)cc1Cl)Nc1nc(NCC2CCCn...,BRD4,0
1,175320120,O=C(Nc1cc(F)cc(F)c1C(=O)O)OCC1c2ccccc2-c2ccccc21,Nc1cccnc1[N+](=O)[O-],Cl.NCC1(OCCO)CCS(=O)(=O)C1,O=C(N[Dy])c1c(F)cc(F)cc1Nc1nc(NCC2(OCCO)CCS(=O...,BRD4,0
2,52014033,Cc1cc(C)c(NC(=O)OCC2c3ccccc3-c3ccccc32)c(C(=O)...,NCc1ccc2[nH]ccc2c1,Nc1ncnc2[nH]ncc12,Cc1cc(C)c(Nc2nc(NCc3ccc4[nH]ccc4c3)nc(Nc3ncnc4...,BRD4,0
3,1091387,C#CC[C@@H](NC(=O)OCC1c2ccccc2-c2ccccc21)C(=O)O,C#CCOc1cccc(CN)c1.Cl,Cc1[nH][nH]c(=O)c1CCN,C#CCOc1cccc(CNc2nc(NCCc3c(C)[nH][nH]c3=O)nc(N[...,sEH,0
4,27979474,COC(=O)c1ccc(C(=O)O)c(NC(=O)OCC2c3ccccc3-c3ccc...,NCC1(OCCO)CCC1,CC(O)(CN)c1ccsc1,COC(=O)c1ccc(C(=O)N[Dy])c(Nc2nc(NCC3(OCCO)CCC3...,HSA,0


In [10]:
df.tail()

,id,buildingblock1_smiles,buildingblock2_smiles,buildingblock3_smiles,molecule_smiles,protein_name,binds
59995,83118851,O=C(NC[C@H]1CC[C@H](C(=O)O)CC1)OCC1c2ccccc2-c2...,CSc1ccc(CN)cc1C#N.Cl,COC(=O)c1c[nH]nc1N,COC(=O)c1c[nH]nc1Nc1nc(NCc2ccc(SC)c(C#N)c2)nc(...,sEH,1
59996,116142642,O=C(N[C@H](CC1CCCC1)C(=O)O)OCC1c2ccccc2-c2ccccc21,Cl.NCc1nc2ccccc2o1,Cl.Cn1cc(N)ccc1=O,Cn1cc(Nc2nc(NCc3nc4ccccc4o3)nc(N[C@H](CC3CCCC3...,BRD4,1
59997,4867498,C#CC[C@H](NC(=O)OCC1c2ccccc2-c2ccccc21)C(=O)O,Cl.Cl.NCc1cncc(F)c1,Cl.NCc1nnc2ncccn12,C#CC[C@H](Nc1nc(NCc2cncc(F)c2)nc(NCc2nnc3ncccn...,HSA,1
59998,56611433,Cc1ccc(C(CC(=O)O)NC(=O)OCC2c3ccccc3-c3ccccc32)cc1,Nc1nc(NCc2ccc(F)cc2)ccc1[N+](=O)[O-],Cc1cc2cc(CN)ccc2[nH]1,Cc1ccc(C(CC(=O)N[Dy])Nc2nc(NCc3ccc4[nH]c(C)cc4...,sEH,1
59999,159174182,O=C(Nc1cc(Br)cc(C(=O)O)c1)OCC1c2ccccc2-c2ccccc21,CCOC(=O)c1ncccc1N,CCOC(=O)c1cc(N)[nH]n1,CCOC(=O)c1cc(Nc2nc(Nc3cc(Br)cc(C(=O)N[Dy])c3)n...,sEH,1


## ECFP encode

In [11]:
from rdkit import Chem
from rdkit.Chem import AllChem
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import average_precision_score
from sklearn.preprocessing import OneHotEncoder

In [12]:
# Convert SMILES to RDKit molecules
df['molecule'] = df['molecule_smiles'].apply(Chem.MolFromSmiles)

# Generate ECFPs
def generate_ecfp(molecule, radius=2, bits=1024): #!!we could change radius and bits
    if molecule is None:
        return None
    return list(AllChem.GetMorganFingerprintAsBitVect(molecule, radius, nBits=bits))

df['ecfp'] = df['molecule'].apply(generate_ecfp)

In [17]:
df.head().iloc[:,-1]

0    [0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, ...
1    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, ...
2    [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ...
3    [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
4    [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, ...
Name: ecfp, dtype: object

In [24]:
len(df.head().iloc[1,-1])

1024

## Train Model

In [43]:
# One-hot encode the protein_name
onehot_encoder = OneHotEncoder(sparse_output=False)
protein_onehot = onehot_encoder.fit_transform(df['protein_name'].values.reshape(-1, 1))

# Combine ECFPs and one-hot encoded protein_name
X = [ecfp + protein for ecfp, protein in zip(df['ecfp'].tolist(), protein_onehot.tolist())]
y = df['binds'].tolist()

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create and train the random forest model
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred_proba = rf_model.predict_proba(X_test)[:, 1]  # Probability of the positive class

# Calculate the mean average precision
map_score = average_precision_score(y_test, y_pred_proba)
print(f"Mean Average Precision (mAP): {map_score:.2f}")

Mean Average Precision (mAP): 0.96


## Test Precision

### Warning!! This part of the code runs over 15mins, not need to run unless want to submit the result to Kaggle.

In [44]:
import os

# Process the test.parquet file chunk by chunk
test_file = '/mnt/isilon/wang_lab/shared/Belka/test.csv'
output_file = '/mnt/isilon/wang_lab/shared/Belka/analysis/leash_tutorial_submission.csv'  # Specify the path and filename for the output file

# Read the test.parquet file into a pandas DataFrame
for df_test in pd.read_csv(test_file, chunksize=100000):

    # Generate ECFPs for the molecule_smiles
    df_test['molecule'] = df_test['molecule_smiles'].apply(Chem.MolFromSmiles)
    df_test['ecfp'] = df_test['molecule'].apply(generate_ecfp)

    # One-hot encode the protein_name
    protein_onehot = onehot_encoder.transform(df_test['protein_name'].values.reshape(-1, 1))

    # Combine ECFPs and one-hot encoded protein_name
    X_test = [ecfp + protein for ecfp, protein in zip(df_test['ecfp'].tolist(), protein_onehot.tolist())]

    # Predict the probabilities
    probabilities = rf_model.predict_proba(X_test)[:, 1]

    # Create a DataFrame with 'id' and 'probability' columns
    output_df = pd.DataFrame({'id': df_test['id'], 'binds': probabilities})

    # Save the output DataFrame to a CSV file
    output_df.to_csv(output_file, index=False, mode='a', header=not os.path.exists(output_file))
